In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model

np.set_printoptions(2)

## Load data

In [2]:
sequence = np.arange(50) / 50
n_features = 5
n_in = len(sequence) // n_features
sequence = sequence.reshape((1, n_in, n_features))

## Model

In [3]:
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_in,n_features)))
model.add(RepeatVector(n_in))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(n_features)))
model.compile(optimizer='adam', loss='mse')

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               42400     
_________________________________________________________________
repeat_vector (RepeatVector) (None, 10, 100)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 100)           80400     
_________________________________________________________________
time_distributed (TimeDistri (None, 10, 5)             505       
Total params: 123,305
Trainable params: 123,305
Non-trainable params: 0
_________________________________________________________________
None


In [4]:
model.fit(sequence, sequence, epochs=300, verbose=0)

In [5]:
yhat = model.predict(sequence, verbose=0)
np.linalg.norm(yhat-sequence)

0.03938526145182411

## Keep Standalone LSTM Encoder

In [6]:
# connect the encoder LSTM as the output layer
encoder_model = Model(inputs=model.inputs, outputs=model.layers[0].output)
# plot_model(encoder_model, show_shapes=True, to_file='lstm_encoder.png')
# get the feature vector for the input sequence
yhat_enc = encoder_model.predict(sequence)

## Findings

If input range is large, the absolute precision will drop. But relative precision according to the value range does not change. So pruning extreme values is necessary.

## References

1. [Machine Learning Mastery: A Gentle Introduction to LSTM Autoencoders](https://machinelearningmastery.com/lstm-autoencoders/)